In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import requests
import re
import plotly.express as px
from ipywidgets import interactive
from IPython.display import display
from bs4 import BeautifulSoup
from pathlib import Path
from io import StringIO

In [2]:
# Show many outputs from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
def get_data_list(data_folder, extension=False):
    github_url = f"https://github.com/munizrodrigo/datathons-time6-dados/tree/main/{data_folder}"
    result = requests.get(github_url)

    soup = BeautifulSoup(result.text, "html.parser")
    csvfiles = soup.find_all(title=re.compile("\.csv$"))
    xlsxfiles = soup.find_all(title=re.compile("\.xlsx$"))

    filename = []
    for file in csvfiles + xlsxfiles:
        name = file.extract().get_text()
        name = Path(name).resolve().stem
        filename.append(name)
    
    return filename

In [4]:
def get_temperature(city):
    github_url = f"https://raw.githubusercontent.com/munizrodrigo/datathons-time6-dados/main/temperatura_sp/{city}.csv"
    github_url = github_url.replace(" ", "%20")
    result = requests.get(github_url)
    csv_string = StringIO(result.text)
    temperature_df = pd.read_csv(csv_string, delimiter=";", encoding='cp1252')
    temperature_df["din_medicao"] = pd.to_datetime(temperature_df["din_medicao"])
    temperature_df = temperature_df[temperature_df["id_varmeteo"] == "TEM_MAX"]
    temperature_df = temperature_df[["din_medicao", "val_medicao"]]
    temperature_df = temperature_df[temperature_df["val_medicao"].notnull()]
    temperature_df = temperature_df.sort_values(by="din_medicao")
    temperature_fig = px.line(temperature_df, x='din_medicao', y="val_medicao")
    display(temperature_df)
    display(temperature_fig)

In [5]:
city_temperature = interactive(get_temperature, city=get_data_list("temperatura_sp"))

display(city_temperature)

interactive(children=(Dropdown(description='city', options=('Ariranha', 'Avare', 'Barra Bonita', 'Barra do Tur…